Zaliczenie

Chojnacki Mateusz

Preferowane jest otworzenie zaliczania w Google Colab ponieważ w notatniku Jupyter występowały błędy.

**Zaliczenie części Wprowadzenie do Sparka**

Zbior danych to oceny filmow. Dane pochodza z projektu https://movielens.org/
Wiecej informacji o danych mozna znalezc pod adresem: http://files.grouplens.org/datasets/movielens/ml-latest-small-README.html

In [68]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [69]:
pip install pyspark_dist_explore

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [70]:
pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [71]:
from pyspark.sql import functions as f

In [72]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('DataFrame_add').master('local[*]').getOrCreate()

data_path = '/'

Do rozważenia: Zalecanym formatem przechowywania danych jest Apache Parquet.
Wiecej o tym formacie danych mozna sie dowiedziec np. tutaj: https://parquet.apache.org/.
W skrocie, tak strona projektu podsumowuje misje stojaca za tym formatem:
"We created Parquet to make the advantages of compressed, efficient columnar data representation available to any project in the Hadoop ecosystem".

Przyjrzyj sie danym i przeanalizuj. 

Sugestie:
1. Z jakich kolumn skladaja sie wczytane powyzej zbiory?
2. Informacje o ilu filmach znajduja sie w dostarczonym zbiorze?
3. Oceny ilu uzytkownikow znajduja sie w zbiorze?
4. Czy w zbiorze znajduja sie braki danych?
5. Ile filmow nie ma ocen? Ktore filmy nie maja ocen?
6. Ktory film ma najlepsza srednia ocen? Jesli jest takich wiele podaj ten z najwieksza liczba glosow.
7. Jaki procent filmow ma tylko maksymalne oceny?
8. Ktory film na najwyzsza minimalna ocene? Jesli jest takich wiele podaj ten z najwieksza liczba glosow.
9. Jaki jest rozklad ocen?
10. Ile jest filmow zaklasyfikowanych jako dokumentalne 'documentary'?
11. Ktory z filmow dokumentalnych z conajmniej 10 glosami ma najwysza srednia ocene?
12. Jakie sa roznice pomiedzy liczba filmow w zbiorze z roku na rok? Zaloz, ze timestamp reprezentuje liczbe sekund od roku 1960.
13. Ile srednio kategorii przypisanych jest do 1 filmu? Ktory film ma najwiecej kategorii (+co to za kategorie)?

In [73]:
# Wczytywanie danych
links = spark.read.option("header","true").csv("links.csv")
movies = spark.read.option("header","true").csv("movies.csv")
ratings = spark.read.option("header","true").csv("ratings.csv")
tags = spark.read.option("header","true").csv("tags.csv")

In [74]:
# Zadanie 1
# Z jakich kolumn skladaja sie wczytane powyzej zbiory?

print('Kolumny dla links')
links.printSchema()
print('Kolumny dla movies')
movies.printSchema()
print('Kolumny dla ratings')
ratings.printSchema()
print('Kolumny dla tags')
tags.printSchema()

Kolumny dla links
root
 |-- movieId: string (nullable = true)
 |-- imdbId: string (nullable = true)
 |-- tmdbId: string (nullable = true)

Kolumny dla movies
root
 |-- movieId: string (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)

Kolumny dla ratings
root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- timestamp: string (nullable = true)

Kolumny dla tags
root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- tag: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [75]:
# Zadanie 2
# Informacje o ilu filmach znajduja sie w dostarczonym zbiorze?

print('Ile filmów znajduje się w zbiorze ?')
movies.distinct().count()

Ile filmów znajduje się w zbiorze ?


9742

In [76]:
# Zadanie 3
# Oceny ilu uzytkownikow znajduja sie w zbiorze?

print('Oceny ilu użytkowników znajduje się w zbiorze ?')
ratings.select(ratings.userId).distinct().count()

Oceny ilu użytkowników znajduje się w zbiorze ?


610

In [77]:
# Zadanie 4
# Czy w zbiorze znajduja sie braki danych?

print("Braki danych dla links.csv")
links.filter(links.movieId.isNull()).show()
links.filter(links.imdbId.isNull()).show()
links.filter(links.tmdbId.isNull()).show()
print("Braki danych w kolumnie tmdbId")

print("Braki danych dla movies.csv")
movies.filter(movies.movieId.isNull()).show()
movies.filter(movies.title.isNull()).show()
movies.filter(movies.genres.isNull()).show()

print("Braki danych dla ratings.csv")
ratings.filter(ratings.userId.isNull()).show()
ratings.filter(ratings.movieId.isNull()).show()
ratings.filter(ratings.rating.isNull()).show()
ratings.filter(ratings.timestamp.isNull()).show()

print("Braki danych dla tags.csv")
tags.filter(tags.userId.isNull()).show()
tags.filter(tags.movieId.isNull()).show()
tags.filter(tags.tag.isNull()).show()
tags.filter(tags.timestamp.isNull()).show()

Braki danych dla links.csv
+-------+------+------+
|movieId|imdbId|tmdbId|
+-------+------+------+
+-------+------+------+

+-------+------+------+
|movieId|imdbId|tmdbId|
+-------+------+------+
+-------+------+------+

+-------+-------+------+
|movieId| imdbId|tmdbId|
+-------+-------+------+
|    791|0113610|  null|
|   1107|0102336|  null|
|   2851|0081454|  null|
|   4051|0056600|  null|
|  26587|0092337|  null|
|  32600|0377059|  null|
|  40697|0105946|  null|
|  79299|0874957|  null|
+-------+-------+------+

Braki danych w kolumnie tmdbId
Braki danych dla movies.csv
+-------+-----+------+
|movieId|title|genres|
+-------+-----+------+
+-------+-----+------+

+-------+-----+------+
|movieId|title|genres|
+-------+-----+------+
+-------+-----+------+

+-------+-----+------+
|movieId|title|genres|
+-------+-----+------+
+-------+-----+------+

Braki danych dla ratings.csv
+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
+------+-

In [78]:
# Zadanie 5
# Ile filmow nie ma ocen? Ktore filmy nie maja ocen?

join_movies_task5 = links.join(other=movies, on='movieId', how='inner')
print('Jakie filmy nie mają ocen ?')
join_movies_task5.select('title', 'tmdbId').filter(join_movies_task5.tmdbId.isNull()).show()
print('Ile filmów nie ma ocen ?')
join_movies_task5.select('title', 'tmdbId').filter(join_movies_task5.tmdbId.isNull()).count()

Jakie filmy nie mają ocen ?
+--------------------+------+
|               title|tmdbId|
+--------------------+------+
|Last Klezmer: Leo...|  null|
|        Loser (1991)|  null|
|     Saturn 3 (1980)|  null|
|Horrors of Spider...|  null|
|Decalogue, The (D...|  null|
|         Eros (2004)|  null|
|           Babylon 5|  null|
|No. 1 Ladies' Det...|  null|
+--------------------+------+

Ile filmów nie ma ocen ?


8

In [79]:
# Zadanie 6
# Ktory film ma najlepsza srednia ocen? Jesli jest takich wiele podaj ten z najwieksza liczba glosow.

movies_ratings_task6 = movies.join(other=ratings, on='movieId', how='inner')

movie_ratings = movies_ratings_task6.\
groupBy('title').\
agg(f.mean('rating').alias('srednia'),\
    f.count('userId').alias('liczba_ocen')).\
orderBy('liczba_ocen')

print('Najwyższa średnia danego filmu dla ocen większej niż 150')
movie_ratings.\
select('title', 'srednia', 'liczba_ocen').\
where(movie_ratings.liczba_ocen > 150).\
sort(movie_ratings.srednia.desc()).show()

Najwyższa średnia danego filmu dla ocen większej niż 150
+--------------------+------------------+-----------+
|               title|           srednia|liczba_ocen|
+--------------------+------------------+-----------+
|Shawshank Redempt...| 4.429022082018927|        317|
|Godfather, The (1...|         4.2890625|        192|
|   Fight Club (1999)| 4.272935779816514|        218|
|Usual Suspects, T...| 4.237745098039215|        204|
|Star Wars: Episod...| 4.231075697211155|        251|
|Schindler's List ...|             4.225|        220|
|Star Wars: Episod...|4.2156398104265405|        211|
|Raiders of the Lo...|            4.2075|        200|
| Pulp Fiction (1994)| 4.197068403908795|        307|
|  Matrix, The (1999)| 4.192446043165468|        278|
| Forrest Gump (1994)| 4.164133738601824|        329|
|Silence of the La...| 4.161290322580645|        279|
|Saving Private Ry...|4.1462765957446805|        188|
|Star Wars: Episod...| 4.137755102040816|        196|
|      Memento (2000)| 4.

In [80]:
# Zadanie 7
# Jaki procent filmow ma tylko maksymalne oceny?

numberOfFilms = movies.select('movieId').count()
print('Liczba wszystkich filmów :' + str(numberOfFilms))


join_movie_rating = movies.join(other = ratings, on = 'movieId', how = 'inner')
numbers_movie_max_rating = join_movie_rating.\
select('movieId').\
where(ratings.rating == 5.0).distinct().count()
print('Liczba filmów z max oceną : ' + str(numbers_movie_max_rating))


procent = (numbers_movie_max_rating / numberOfFilms)*100
print('Procent maksymalnych ocen względem wszystkich ocen :' + str(procent) +' %')

Liczba wszystkich filmów :9742
Liczba filmów z max oceną : 2954
Procent maksymalnych ocen względem wszystkich ocen :30.322315746253338 %


In [81]:
# Zadanie 8
# Ktory film ma najwyzsza minimalna ocene? Jesli jest takich wiele podaj ten z najwieksza liczba glosow.

sort_mark = ratings.\
groupBy('movieId', 'rating').\
agg(f.count('rating').alias('Suma_ocen'))

join_movie_rating2 = movies.join(other = sort_mark, on = 'movieId', how = 'inner')

print('--------- Wynik ---------')

film_rating_min = join_movie_rating2.\
select('title', 'rating', 'Suma_ocen').\
orderBy(sort_mark.rating.desc(),\
        sort_mark.Suma_ocen.desc()).show()

--------- Wynik ---------
+--------------------+------+---------+
|               title|rating|Suma_ocen|
+--------------------+------+---------+
|Shawshank Redempt...|   5.0|      153|
| Pulp Fiction (1994)|   5.0|      123|
| Forrest Gump (1994)|   5.0|      116|
|  Matrix, The (1999)|   5.0|      109|
|Star Wars: Episod...|   5.0|      104|
|Silence of the La...|   5.0|       92|
|Schindler's List ...|   5.0|       92|
|Godfather, The (1...|   5.0|       88|
|   Fight Club (1999)|   5.0|       81|
|Star Wars: Episod...|   5.0|       80|
|   Braveheart (1995)|   5.0|       80|
|Usual Suspects, T...|   5.0|       71|
|Raiders of the Lo...|   5.0|       69|
|American Beauty (...|   5.0|       65|
|Star Wars: Episod...|   5.0|       65|
|Terminator 2: Jud...|   5.0|       64|
|Lord of the Rings...|   5.0|       62|
|Saving Private Ry...|   5.0|       61|
|        Fargo (1996)|   5.0|       57|
|Princess Bride, T...|   5.0|       55|
+--------------------+------+---------+
only showing t

In [82]:
# Zadanie 9
# Jaki jest rozklad ocen?

import matplotlib.pyplot as plt

print('--------- Rozkład ocen ---------')
x = ratings.groupBy('rating').count().orderBy(ratings.rating).show()

--------- Rozkład ocen ---------
+------+-----+
|rating|count|
+------+-----+
|   0.5| 1370|
|   1.0| 2811|
|   1.5| 1791|
|   2.0| 7551|
|   2.5| 5550|
|   3.0|20047|
|   3.5|13136|
|   4.0|26818|
|   4.5| 8551|
|   5.0|13211|
+------+-----+



In [83]:
# Zadanie 10
# Ile jest filmow zaklasyfikowanych jako dokumentalne 'documentary'?

# movies.show()

how_many_documentary = movies.\
select('title', 'genres').\
where(f.col('genres').like('%Documentary%')).\
count()

print(f'Filmów zaklasyfikowanych jako Documentary w danej bazie jest : ' + str(how_many_documentary))

Filmów zaklasyfikowanych jako Documentary w danej bazie jest : 440


In [84]:
# Zadanie 11
# Ktory z filmow dokumentalnych z conajmniej 10 glosami ma najwysza srednia ocene?

count_votes = ratings.\
groupBy('movieId').\
agg(f.count('movieId').alias('Suma_ocen'))

votes_more_10 = count_votes.\
select('movieId','Suma_ocen').\
where(count_votes.Suma_ocen > 9)

join_movie_votes = ratings.join(other = votes_more_10, on = 'movieId', how = 'inner')

mean_film_rating = join_movie_votes.\
groupBy('movieId').\
agg(f.mean('rating').alias('Srednia'))

join_movie_final = movies.join(other = mean_film_rating, on = 'movieId', how = 'inner')

print('Najlepszy film pod względem średniej oceny gdzie liczba głosów wynosiła min 10')

title_best_mean = join_movie_final.\
select('title', 'Srednia').\
orderBy(mean_film_rating.Srednia.desc()).show()


Najlepszy film pod względem średniej oceny gdzie liczba głosów wynosiła min 10
+--------------------+------------------+
|               title|           Srednia|
+--------------------+------------------+
|Secrets & Lies (1...| 4.590909090909091|
|Guess Who's Comin...| 4.545454545454546|
|Paths of Glory (1...| 4.541666666666667|
|Streetcar Named D...|             4.475|
|Celebration, The ...| 4.458333333333333|
|          Ran (1985)| 4.433333333333334|
|Shawshank Redempt...| 4.429022082018927|
|His Girl Friday (...| 4.392857142857143|
|All Quiet on the ...|              4.35|
| Hustler, The (1961)| 4.333333333333333|
|Sunset Blvd. (a.k...| 4.333333333333333|
|Double Indemnity ...| 4.323529411764706|
|It Happened One N...| 4.321428571428571|
|Philadelphia Stor...| 4.310344827586207|
|Fog of War: Eleve...|4.3076923076923075|
|Living in Oblivio...|4.3076923076923075|
|Once Upon a Time ...| 4.305555555555555|
|In the Name of th...|               4.3|
|Elite Squad (Trop...|               4.

In [85]:
# Zadanie 13
# Ile srednio kategorii przypisanych jest do 1 filmu? Ktory film ma najwiecej kategorii (+co to za kategorie)?

df_new = movies.withColumn('category', f.split('genres', '\|')) \
           .withColumn('cnt', f.expr('size(filter(category, x -> x in ("Adventure","Comedy","Drama","Crime","Horror","Animation", "Children", "Comedy", "Fantasy", "Romance", "Action", "Thriller", "Mystery", "Sci-Fi", "War", "Musical", "Documentary", "Musical", "Western", "Film-Noir")))'))


all_films = df_new.count()
sum_categories = df_new.agg(f.sum("cnt")).collect()[0][0]
mean_film_cat = sum_categories/all_films
print(f'Średnia liczba kategorii na film : ' + str(mean_film_cat))

print(f'===========================================================')

max_cat = df_new.orderBy(df_new.cnt.desc())

print(f'Film z największą liczbą kategorii')
max_cat.show(truncate=False)

# Jakie kategorie ?
# my_list = movies.select(f.collect_list('genres')).first()[0]
# my_list


Średnia liczba kategorii na film : 2.2471771710121127
Film z największą liczbą kategorii
+-------+-------------------------------------------------------------------------------+-----------------------------------------------------------------------------+----------------------------------------------------------------------------------------+---+
|movieId|title                                                                          |genres                                                                       |category                                                                                |cnt|
+-------+-------------------------------------------------------------------------------+-----------------------------------------------------------------------------+----------------------------------------------------------------------------------------+---+
|81132  |Rubber (2010)                                                                  |Action|Adventure|Comedy|Crime|Drama|Fil